In [1]:
from imp import reload
import utils as ul
from mlr import Model as MD
import fw
#import park as sc
import numpy as np

## Data generation

In [20]:
## X: n*p  W: p*m  Y: n*m
np.random.seed(0)
X, y, W = MD.generate(n = 16)
n = X.shape[0]
m = W.shape[1]
p = X.shape[1]

In [3]:
from sklearn.linear_model import MultiTaskLasso
MultiTaskLasso(alpha=.001).fit(X, y).score(X, y)

0.55115487849365286

In [34]:
points = ul.mat2point(X,y)
dataRDD = sc.parallelize(points,8).mapPartitions(ul.point2mat)

In [35]:
dataRDD.first()

[array([[ 1.76405235,  0.40015721,  0.97873798,  2.2408932 ],
        [ 1.86755799, -0.97727788,  0.95008842, -0.15135721]]),
 array([[0, 0, 0, 1, 0],
        [1, 0, 0, 0, 0]])]

In [36]:
dataRDD = dataRDD.map(fw.row_data).setName('dataRDD').persist()
dataRDD

dataRDD PythonRDD[27] at RDD at PythonRDD.scala:43

In [37]:
dataRDD.first()

{'X': array([[ 1.76405235,  0.40015721,  0.97873798,  2.2408932 ],
        [ 1.86755799, -0.97727788,  0.95008842, -0.15135721]]),
 'Y': array([[0, 0, 0, 1, 0],
        [1, 0, 0, 0, 0]]),
 'm': 5,
 'n': 2,
 'p': 4}

In [25]:
def score2class(z):
    z['Y'] = MD.score2class(z['Y'])
    return z

In [26]:
dataRDD = dataRDD.map(score2class).setName('dataRDD').persist()
dataRDD

dataRDD PythonRDD[18] at RDD at PythonRDD.scala:43

In [27]:
dataRDD.first()

{'X': array([[ 1.76405235,  0.40015721,  0.97873798,  2.2408932 ],
        [ 1.86755799, -0.97727788,  0.95008842, -0.15135721]]),
 'Y': array([3, 0]),
 'm': 5,
 'n': 2,
 'p': 4}

## Initialization

In [38]:
np.random.seed(0)

T = 30               # number of iterations of FW
U = np.zeros((T,p))  # left singular vector
V = np.zeros((T,m))  # right singular vector

loss = np.zeros(T)

In [39]:
rdd = dataRDD.map(fw.row_param)
rdd

PythonRDD[29] at RDD at PythonRDD.scala:43

In [40]:
rdd.first()

{'W': array([[ 0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.]]),
 'X': array([[ 1.76405235,  0.40015721,  0.97873798,  2.2408932 ],
        [ 1.86755799, -0.97727788,  0.95008842, -0.15135721]]),
 'Y': array([[0, 0, 0, 1, 0],
        [1, 0, 0, 0, 0]]),
 'm': 5,
 'n': 2,
 'p': 4,
 't': 0}

## Subfunctions

In [41]:
rdd = fw.gradient(rdd, MD)
rdd

withGradient PythonRDD[31] at RDD at PythonRDD.scala:43

In [43]:
rdd.first()['grad']

array([[-1.14123592,  0.72632207,  0.72632207, -1.03773028,  0.72632207],
       [ 0.86185375, -0.11542413, -0.11542413, -0.51558134, -0.11542413],
       [-0.56432314,  0.38576528,  0.38576528, -0.5929727 ,  0.38576528],
       [ 0.56926441,  0.4179072 ,  0.4179072 , -1.822986  ,  0.4179072 ]])

In [44]:
fw.centralize(rdd)

(array([ 1.        ,  0.20060436,  0.354374  , -0.1545394 ]),
 array([-0.70897714,  0.03078158,  1.        , -0.73329464,  0.41149019]))

In [45]:
fw.warmstart(rdd).first()

{'W': array([[ 0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.]]),
 'X': array([[ 1.76405235,  0.40015721,  0.97873798,  2.2408932 ],
        [ 1.86755799, -0.97727788,  0.95008842, -0.15135721]]),
 'Y': array([[0, 0, 0, 1, 0],
        [1, 0, 0, 0, 0]]),
 'grad': array([[-1.14123592,  0.72632207,  0.72632207, -1.03773028,  0.72632207],
        [ 0.86185375, -0.11542413, -0.11542413, -0.51558134, -0.11542413],
        [-0.56432314,  0.38576528,  0.38576528, -0.5929727 ,  0.38576528],
        [ 0.56926441,  0.4179072 ,  0.4179072 , -1.822986  ,  0.4179072 ]]),
 'm': 5,
 'n': 2,
 'p': 4,
 't': 0,
 'u': array([ 1.        ,  0.06239289,  0.54471949,  0.97406035]),
 'v': array([-0.26521556,  0.42173852,  0.42173852, -1.        ,  0.42173852])}

In [48]:
fw.avgmix(rdd)

(array([ 5.47157215,  1.31256375,  2.44970703,  1.55202166]),
 array([-2.62817233,  0.18330323,  2.31430016, -2.7032532 ,  2.83382214]))

In [49]:
fw.poweriter(rdd, lambda t: 10)

(array([ 0.91681161,  0.18392373,  0.32489397, -0.14167726]),
 array([-0.47683778,  0.02070286,  0.67257329, -0.49319938,  0.27676101]))

In [ ]:
np.array([fw.loground(i) for i in range(101)])

In [50]:
u, v = fw.centralize(rdd)
u, v

(array([ 1.        ,  0.20060436,  0.354374  , -0.15453941]),
 array([-0.70897714,  0.03078158,  1.        , -0.73329464,  0.41149019]))

In [51]:
u, v = fw.regularize(u, v, 1)
u, v

(array([-0.91681196, -0.18391648, -0.32489432,  0.14168358]),
 array([-0.47684045,  0.02070293,  0.67257522, -0.4931958 ,  0.27675811]))

In [52]:
rdd = fw.broadcast(rdd, u, v)
rdd.first()['D']

array([[ 0.43717303, -0.01898069, -0.616625  ,  0.45216781, -0.25373514],
       [ 0.08769882, -0.00380761, -0.12369767,  0.09070684, -0.05090038],
       [ 0.15492276, -0.00672626, -0.21851587,  0.16023652, -0.08991714],
       [-0.06756046,  0.00293326,  0.09529286, -0.06987774,  0.03921208]])

In [53]:
np.outer(-u,v)

array([[-0.43717303,  0.01898069,  0.616625  , -0.45216781,  0.25373514],
       [-0.08769882,  0.00380761,  0.12369767, -0.09070684,  0.05090038],
       [-0.15492276,  0.00672626,  0.21851587, -0.16023652,  0.08991714],
       [ 0.06756046, -0.00293326, -0.09529286,  0.06987774, -0.03921208]])

In [54]:
fw.loss(rdd, MD)

25.751006598945601

In [55]:
rdd, _ = fw.naivestep(rdd)
rdd.first()['a']

1.0

In [56]:
rdd = fw.descent(rdd)
rdd.first()['W']

array([[ 0.43717303, -0.01898069, -0.616625  ,  0.45216781, -0.25373514],
       [ 0.08769882, -0.00380761, -0.12369767,  0.09070684, -0.05090038],
       [ 0.15492276, -0.00672626, -0.21851587,  0.16023652, -0.08991714],
       [-0.06756046,  0.00293326,  0.09529286, -0.06987774,  0.03921208]])

In [57]:
fw.loss(rdd, MD)

18.099751803048473

In [58]:
rdd = fw.ascent(rdd)
rdd.first()['W']

array([[ 0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.]])

In [59]:
fw.loss(rdd, MD)

25.751006598945601

## Main function

In [60]:
np.random.seed(0)
rdd = dataRDD.map(fw.row_param)
fw.iterate(rdd, MD, fw.centralize, fw.naivestep, 1, 0)

(PythonRDD[72] at RDD at PythonRDD.scala:43,
 (array([-0.91681196, -0.18391648, -0.32489432,  0.14168357]),
  array([-0.47684045,  0.02070293,  0.67257522, -0.4931958 ,  0.27675811])),
 None,
 18.099751803138894,
 7)

In [61]:
np.random.seed(0)
rdd = dataRDD.map(fw.row_param)
fw.iterate(rdd, MD, fw.avgmix, fw.naivestep, 1, 0)

(PythonRDD[78] at RDD at PythonRDD.scala:43,
 (array([-0.86436731, -0.20735123, -0.38699054, -0.2451794 ]),
  array([-0.49994715,  0.03486907,  0.44024045, -0.5142295 ,  0.53906713])),
 None,
 19.027685183201619,
 9)

In [62]:
np.random.seed(0)
rdd = dataRDD.map(fw.row_param)
fw.iterate(rdd, MD, lambda x,t: fw.poweriter(x,lambda t: 1), fw.naivestep, 1, 0)

(PythonRDD[86] at RDD at PythonRDD.scala:43,
 (array([-0.91162895, -0.26262519, -0.31485919,  0.02871159]),
  array([-0.44092348,  0.02460662,  0.64077952, -0.54187703,  0.31741437])),
 None,
 18.201043907471231,
 8)

In [63]:
np.random.seed(0)
rdd = dataRDD.map(fw.row_param)
fw.iterate(rdd, MD, lambda x,t: fw.poweriter(x,lambda t: 10), fw.naivestep, 1, 0)

(PythonRDD[112] at RDD at PythonRDD.scala:43,
 (array([-0.91681161, -0.18392373, -0.32489397,  0.14167726]),
  array([-0.47683778,  0.02070286,  0.67257329, -0.49319938,  0.27676101])),
 None,
 18.099755666207873,
 7)

In [64]:
np.random.seed(0)
rdd = dataRDD.map(fw.row_param)
fw.iterate(rdd, MD, lambda x,t: fw.poweriter(x,fw.loground), fw.naivestep, 1, 0)

(PythonRDD[120] at RDD at PythonRDD.scala:43,
 (array([-0.91163117, -0.26261564, -0.31485988,  0.02872097]),
  array([-0.44092755,  0.02460666,  0.64078273, -0.54187225,  0.31741041])),
 None,
 18.201031480768425,
 8)

In [65]:
np.random.seed(0)
v0 = np.random.randn(m)
np.random.seed(0)
rdd = dataRDD.map(fw.row_param)
fw.iterate(rdd, MD, lambda x,t: fw.poweriter(x,fw.loground,v0), fw.naivestep, 1, 0)

(PythonRDD[127] at RDD at PythonRDD.scala:43,
 (array([ 0.93695611,  0.04651203,  0.34566378, -0.0215969 ]),
  array([ 0.50042843, -0.04885705, -0.66216079,  0.48381387, -0.27322445])),
 None,
 18.259712499964124,
 8)